<a href="https://colab.research.google.com/github/SuveenE/CG-4002/blob/main/ai/software/load_and_test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies if needed
!pip install pandas numpy scikit-learn tensorflow

LABEL_MAP = {
"NONE": 0,
"SLIDE_LEFT": 1,
"SLIDE_RIGHT": 2,
"WRIST_TURN_CLOCKWISE": 3,
"WRIST_TURN_ANTI_CLOCKWISE": 4,
"SLIDE_UP": 5,
"SLIDE_DOWN": 6,
"GRASP": 7
}

In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import collections
from google.colab import drive

# Now mount cleanly
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reconstruct the MLP Model and Scalar


In [3]:
import tensorflow.keras as keras
from tensorflow.keras import layers

# Define the input dimension
input_dim = 84
num_classes = 8 # Based on the previous notebook execution, 8 classes were identified.

# Create the Keras MLP model
inputs = keras.Input(shape=(input_dim,), name="features", dtype='float32')
x = layers.Dense(64, activation="relu", name="dense_64", dtype='float32')(inputs)
x = layers.Dense(32, activation="relu", name="dense_32", dtype='float32')(x)
logits = layers.Dense(num_classes, activation=None, name="logits", dtype='float32')(x)

reconstructed_model = keras.Model(inputs=inputs, outputs=logits, name="imu_mlp_84_64_32")
reconstructed_model.summary()

Model: "imu_mlp_84_64_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         5,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,784 (30.41 KB)

 Trainable params: 7,784 (30.41 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
import os
import numpy as np
from pathlib import Path

sub_folder_name = "segmented_features"
OUT_DIR = Path(f"/content/drive/My Drive/cg4002/test/2-imu-model/{sub_folder_name}")
OUT_DIR.mkdir(parents=True, exist_ok=True)
WEIGHTS_DIR = OUT_DIR / "weights_npy"
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# Load weights into the reconstructed model
for lyr in reconstructed_model.layers:
    if isinstance(lyr, keras.layers.Dense):
        name = lyr.name
        weights_path = os.path.join(WEIGHTS_DIR, f"{name}_W.npy")
        bias_path = os.path.join(WEIGHTS_DIR, f"{name}_b.npy")

        if os.path.exists(weights_path) and os.path.exists(bias_path):
            W = np.load(weights_path)
            b = np.load(bias_path)
            lyr.set_weights([W, b])
            print(f"Loaded weights for layer: {name} from {weights_path}")
        else:
            print(f"Warning: Weights or biases not found for layer: {name}")

print("All available weights loaded into reconstructed_model.")

Loaded weights for layer: dense_64 from /content/drive/My Drive/cg4002/test/2-imu-model/segmented_features/weights_npy/dense_64_W.npy
Loaded weights for layer: dense_32 from /content/drive/My Drive/cg4002/test/2-imu-model/segmented_features/weights_npy/dense_32_W.npy
Loaded weights for layer: logits from /content/drive/My Drive/cg4002/test/2-imu-model/segmented_features/weights_npy/logits_W.npy
All available weights loaded into reconstructed_model.


In [5]:
import joblib


scaler_path = os.path.join(OUT_DIR, "scaler.pkl")

if os.path.exists(scaler_path):
    scaler = joblib.load(scaler_path)
    print(f"Scaler loaded from: {scaler_path}")
else:
    print(f"Error: Scaler file not found at {scaler_path}")


Scaler loaded from: /content/drive/My Drive/cg4002/test/2-imu-model/segmented_features/scaler.pkl


## Test with 84 features before scaling

In [56]:
import numpy as np

idx_to_raw = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}

# User provided input as a 1D numpy array
# This array has 84 features, corresponding to the model's input_dim
input_array = np.array([
398.21794871794873,532.6621974581268,665.0613139268073,1283.0,302.0,-217.19230769230768,362.1103053962448,422.25155037592435,1229.0,-208.5,80.34615384615384,811.2279838097156,815.1971210411909,1331.0,384.5,2197.1794871794873,7688.512776302976,7996.300795383851,25012.0,320.0,-6245.551282051282,10219.491831722955,11976.849507085935,12608.0,-2690.5,1676.1410256410256,8078.360051466968,8250.415132523596,25012.0,-313.0,1050.53645818805,426.0544792024571,1133.644242798758,2515.3401360452226,1024.136213174274,12709.719078950613,10673.244179666,16596.840072273255,37552.123135716305,7776.465603037326,395.34615384615387,699.171026121119,803.2052696093353,1999.0,451.0,-63.92307692307692,527.4881155358462,531.3472233811633,1445.0,63.0,-337.3205128205128,639.9965042690937,723.4505192797596,1999.0,-271.0,3181.0,7542.990803150316,8186.2977747215045,25012.0,1431.5,-5971.397435897436,12840.645632721185,14161.206431747405,25012.0,-5819.0,-1405.179487179487,9051.721389969453,9160.141347862516,13770.0,110.0,1081.478785426363,530.3327893332163,1204.512030147159,2832.5883922659855,1002.0551154006073,14797.731204067246,11510.455229223966,18747.357903763263,43322.632156876156,10211.916852884846,0.60268734369322,0.8973101330628042,1.0625132512227815,1.1295739322741045
                         ])
input_features = input_array.reshape(1, -1)
scaled_input_features = scaler.transform(input_features)

predictions = reconstructed_model.predict(scaled_input_features, verbose=0)

# Define softmax function
def softmax(z):
    z = z - np.max(z)
    ez = np.exp(z)
    return ez / np.sum(ez)

probs = softmax(predictions[0])
top = int(np.argmax(probs))

# Print the softmax probabilities and top class
print(f"AI prediction probabilities: {probs}")
print(f"Top class: {top}, confidence: {float(probs[top]):.4f}")

predicted_label_idx = np.argmax(predictions, axis=1)[0]

# Map the predicted index back to the raw label
predicted_raw_label = idx_to_raw.get(predicted_label_idx, f"Unknown index: {predicted_label_idx}")

# Print the results
print(f"Input Feature Vector (first 5 values): {input_features[0,:5]}...")
csv_string = '\n'.join(
    [', '.join(map(str, row)) for row in scaled_input_features]
)

print(f"Scaled Input Feature Vector:\n{csv_string}")
print(f"Raw Logit Predictions: {predictions[0]}")
print(f"Predicted Label Index: {predicted_label_idx}")
print(f"Predicted Raw Label: {predicted_raw_label}")

# Optional: Print the friendly label name using the LABEL_MAP
friendly_label_found = False
for name, idx in LABEL_MAP.items():
    if idx == predicted_raw_label:
        print(f"Predicted Friendly Label: {name}")
        friendly_label_found = True
        break
if not friendly_label_found:
    print("Could not find a friendly name for the predicted label in LABEL_MAP.")

AI prediction probabilities: [7.18080206e-04 4.65899765e-01 3.21099460e-01 9.89279971e-02
 1.48584571e-04 2.99794786e-03 2.83998135e-03 1.07368246e-01]
Top class: 1, confidence: 0.4659
Input Feature Vector (first 5 values): [ 398.21794872  532.66219746  665.06131393 1283.          302.        ]...
Scaled Input Feature Vector:
0.19089168537537068, 0.5915649810875959, -0.0511560938203284, 0.6842627840198753, 0.07836014841032041, 0.1439257206961311, 0.5474478448768242, -0.4193696401326615, 1.5995801670196588, 0.15591460379853594, -0.0030314412515751216, 1.0174425001786898, 0.1669415888104223, 0.9442611272855594, 0.37286225523099004, 0.29116796281595825, 0.3887764164371733, 0.08649503351544047, 1.4510948231579268, 0.014725851291980905, -0.7688362768161859, 0.5440402738236971, 0.41071457442435066, 0.2528082668353363, -0.39538506097931536, 0.24807705958319995, 0.49092188307300716, 0.18952287881851346, 1.7046652311809014, -0.13605891772249779, -0.14083640125519653, 0.614461225480739, 0.068181

## Test with csv file with processed features

In [12]:
import numpy as np
import pandas as pd
import os

# Re-define LABEL_MAP and idx_to_raw if not available from previous cells
LABEL_MAP = {
    "NONE": 0,
    "SLIDE_LEFT": 1,
    "SLIDE_RIGHT": 2,
    "WRIST_TURN_CLOCKWISE": 3,
    "WRIST_TURN_ANTI_CLOCKWISE": 4,
    "SLIDE_UP": 5,
    "SLIDE_DOWN": 6,
    "GRASP": 7
}

idx_to_raw = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}

# Read the CSV file
csv_path = os.path.join(OUT_DIR, "segmented-test-data-2.csv")  # This path should contain only features
df = pd.read_csv(csv_path)

# Assume all columns are features, as there are no labels
input_features_raw = df.values

# Scale the input features using the pre-loaded scaler
scaled_input_features = scaler.transform(input_features_raw)

print("scaled", scaled_input_features[0])

predictions = reconstructed_model.predict(scaled_input_features, verbose=0)

def softmax(z):
    z = z - np.max(z)
    ez = np.exp(z)
    return ez / np.sum(ez)

print("="*80)
print(f"Processing {len(df)} samples from {csv_path}")
print("="*80)

for i in range(len(df)):
    logits = predictions[i]               # raw model output
    probs = softmax(logits)              # convert to probabilities
    if i == 0:
      print("logits", logits)
      print("probs", probs)
    top_idx = int(np.argmax(probs))      # predicted class index
    confidence = float(probs[top_idx])   # probability value

    predicted_raw_label = idx_to_raw.get(top_idx, "Unknown")

    # Find friendly name for predicted label
    predicted_friendly = next((name for name, idx in LABEL_MAP.items() if idx == predicted_raw_label), "Unknown")

    print(f"{i+1}. Predicted Index={top_idx}, Predicted Class={predicted_raw_label}, Predicted Label={predicted_friendly}, Confidence={confidence:.4f}")

scaled [ 0.25402859 -0.6033972  -0.90844758 -0.48977445  0.16744792 -0.73360965
 -0.42779817 -0.07398879 -0.93562148 -0.7052804   0.66089827 -0.5942818
 -0.6322923   0.32311039  0.51061566  0.06079605 -0.49884    -1.09757004
 -0.72211242  0.1489465   0.08394026 -0.8389402  -1.60116954 -0.55528387
  0.09617736 -0.16948007 -0.45997017 -1.08146477 -0.54081413 -0.30627879
 -0.43787842 -0.62790648 -0.48636233 -1.11863917 -0.45128257 -1.37878776
 -1.81780586 -1.89897888 -2.28685896 -0.94320665  0.50431027 -0.95862551
 -0.39181005 -0.18405211  0.38876504 -0.24372716 -0.75603872 -1.35580212
 -1.23532305 -0.17610816  0.77814003 -1.40687079 -1.65417793 -0.74985953
  0.55126676 -0.08673608 -0.75189759 -1.55920877 -1.34373086 -0.18726913
 -0.26300535 -1.13964987 -2.21798203 -1.29136062 -0.12257833 -0.0710041
 -0.51245024 -1.22398843 -0.5288523  -0.22135548 -1.0398436  -0.95337928
 -1.15843198 -1.54852693 -0.85786356 -1.69611149 -1.90134427 -2.2837733
 -2.54274748 -1.29004751  1.13214709  0.6497520

In [7]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, classification_report

# Re-define LABEL_MAP and idx_to_raw if not available from previous cells
LABEL_MAP = {
    "NONE": 0,
    "SLIDE_LEFT": 1,
    "SLIDE_RIGHT": 2,
    "WRIST_TURN_CLOCKWISE": 3,
    "WRIST_TURN_ANTI_CLOCKWISE": 4,
    "SLIDE_UP": 5,
    "SLIDE_DOWN": 6,
    "GRASP": 7
}

idx_to_raw = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}

# Read the CSV file
csv_path = os.path.join(OUT_DIR, "segmented-test-data-2.csv")  # Update this path as needed
df = pd.read_csv(csv_path)

# Extract true labels (3rd column, index 2) and features
true_labels = df.iloc[:, 2].values.astype(int)
# Drop the first two columns (metadata) and the true label column
input_features_raw = df.iloc[:, 3:].values

# Scale the input features
scaled_input_features = scaler.transform(input_features_raw)

predictions = reconstructed_model.predict(scaled_input_features, verbose=0)

def softmax(z):
    z = z - np.max(z)
    ez = np.exp(z)
    return ez / np.sum(ez)

print("="*80)
print(f"Processing {len(df)} samples from {csv_path}")
print("="*80)

predicted_labels_list = []

for i in range(len(df)):
    logits = predictions[i]               # raw model output
    probs = softmax(logits)              # convert to probabilities
    top_idx = int(np.argmax(probs))      # predicted class index
    confidence = float(probs[top_idx])   # probability value

    predicted_raw_label = idx_to_raw.get(top_idx, "Unknown")
    true_raw_label = true_labels[i] # Get the true label for this sample

    # Find friendly name for predicted label
    predicted_friendly = next((name for name, idx in LABEL_MAP.items() if idx == predicted_raw_label), "Unknown")
    # Find friendly name for true label
    true_friendly = next((name for name, idx in LABEL_MAP.items() if idx == true_raw_label), "Unknown")

    print(f"{i+1}. Pred_idx={top_idx} Pred_cls={predicted_raw_label} Pred_label={predicted_friendly} Conf={confidence:.4f} | True_cls={true_raw_label} True_label={true_friendly}")
    predicted_labels_list.append(predicted_raw_label)


# Convert predicted labels to a numpy array
predicted_labels = np.array(predicted_labels_list)

# Generate Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("\n" + "="*80)
print("Confusion Matrix:")
print(cm)

# Generate Classification Report
report = classification_report(true_labels, predicted_labels, target_names=[name for name, idx in sorted(LABEL_MAP.items(), key=lambda item: item[1])], zero_division=0)
print("\n" + "="*80)
print("Classification Report:")
print(report)

ValueError: X has 81 features, but StandardScaler is expecting 84 features as input.